# HW7 DATASCI W261: Machine Learning at Scale  
 
* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  11
* **Homework Number:**  10
* **Date:**  7/28/16

### Create SparkContext

In [33]:
import pyspark
from pyspark.sql import SQLContext

# We can give a name to our app (to find it in Spark WebUI) and configure execution mode
# In this case, it is local multicore execution with "local[*]"
#app_name = "example-logs"
app_name = "HW10"
master = "local[*]"
conf = pyspark.SparkConf().setAppName(app_name).setMaster(master)
sc.stop()
sc = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(sc)

print sc
print sqlContext

In [ ]:
sc.stop()

# HW 10.1
In Spark write the code to count how often each word appears in a text document (or set of documents). Please use this homework document as a the example document to run an experiment.  Report the following: provide a sorted list of tokens in decreasing order of frequency of occurence.

In [ ]:
#file = sc.textFile("file:some_local_text_file_pathname")
#wordCounts = file.flatMap(lambda line: line.strip().split(" ")) \
#    .map(lambda word: (word, 1)) \
#    .reduceByKey(lambda a, b: a + b, 1) \ # last arg configures one reducer task
#    .map(lambda (a, b): (b, a)) \
#    .sortByKey(1, 1) \ # 1st arg configures ascending sort, 2nd configures 1 task
#    .map(lambda (a, b): (b, a))

In [23]:
from operator import add
lines = sc.textFile('MIDS-MLS-HW-10.txt').flatMap(lambda x: x.split())
print lines.take(5)
print lines.count()
l1 = lines.map(lambda x: (x, 1))
print l1.take(5)
print l1.count()
#l2 = l1.reduceByKey(lambda x, y: x + y)
l2 = l1.reduceByKey(add)
print l2.take(5)
print l2.count()
l3 = l2.map(lambda x: (x[1], x[0]))
print l3.take(10)
print l3.count()
l4 = l3.sortByKey(False)
print l4.take(10)
print l4.count()
l5 = l4.map(lambda x: (x[1], x[0]))
print l5.take(10)
print l5.count()
for v in l5.collect():
    print v[0], v[1]

[u'===========================================================================', u'===MIDS', u'UC', u'Berkeley,', u'Machine']
1001
[(u'===========================================================================', 1), (u'===MIDS', 1), (u'UC', 1), (u'Berkeley,', 1), (u'Machine', 1)]
1001
[(u'y)', 1), (u'comment', 2), (u'What', 4), (u'from', 6), (u'weight(X)=', 1)]
483
[(1, u'y)'), (2, u'comment'), (4, u'What'), (6, u'from'), (1, u'weight(X)='), (1, u'10.2:'), (1, u'=====DATSCIW261'), (1, u'intuitoive'), (1, u'(cluster'), (1, u'consists')]
483
[(70, u'the'), (36, u'and'), (25, u'in'), (24, u'of'), (18, u'a'), (12, u'for'), (12, u'code'), (12, u'to'), (11, u'is'), (11, u'model')]
483
[(u'the', 70), (u'and', 36), (u'in', 25), (u'of', 24), (u'a', 18), (u'for', 12), (u'code', 12), (u'to', 12), (u'is', 11), (u'model', 11)]
483
the 70
and 36
in 25
of 24
a 18
for 12
code 12
to 12
is 11
model 11
with 10
data 10
=== 9
as 9
on 9
plot 9
this 8
= 8
Using 8
your 8
# 7
each 7
KMeans 7
from 6
domain 6
l

In [34]:
from operator import add
wordCount = sc.textFile('MIDS-MLS-HW-10.txt').flatMap(lambda x: x.strip().split()) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(add) \
    .map(lambda (x, y): (y, x)) \
    .sortByKey(False) \
    .map(lambda (x, y): (y, x))
for word, count in wordCount.collect():
    print word, count

the 70
and 36
in 25
of 24
a 18
for 12
code 12
to 12
is 11
model 11
with 10
data 10
=== 9
as 9
on 9
plot 9
this 8
= 8
Using 8
your 8
# 7
each 7
KMeans 7
from 6
domain 6
linear 6
Spark 5
training 5
following 5
it 5
regression 5
==HW 5
clusters 5
What 4
Sum 4
Comment 4
Squared 4
using 4
side 4
HW 4
Set 4
example 4
space 4
Here 3
words 3
+ 3
lazy 3
100 3
count 3
iteration 3
report 3
show 3
results 3
document 3
Within 3
ASSIGNMENT 3
Errors 3
===HW 3
(OPTIONAL) 3
import 3
plots 3
by 3
that 3
an 3
provided 3
after 3
x 3
comment 2
Apache 2
homework 2
every 2
Berkeley, 2
word 2
notebook: 2
plots. 2
-- 2
Learning 2
list 2
run 2
regression. 2
above 2
Report 2
evaluation 2
available 2
https://www.dropbox.com/s/q85t0ytb9apggnh/kmeans_data.txt?dl=0 2
here 2
UC 2
Make 2
Please 2
https://www.dropbox.com/s/3nsthvp8g2rrrdh/EM-Kmeans.ipynb?dl=0 2
use 2
RIDGE 2
10.6.1 2
you 2
(using 2
surface 2
function 2
MSE 2
WSSSE 2
HW10.3. 2
Machine 2
LASS0 2
NOTE 2
1 2
set. 2
differences 2
label 2
original 2
heatmap 

# HW 10.1.1
Modify the above word count code to count words that begin with lower case letters (a-z) and report your findings. Again sort the output words in decreasing order of frequency.

In [35]:
from operator import add
import re
def filterLowerCase(s):
    pattern = re.compile("[a-z]")
    return bool(pattern.match(s))

wordCount = sc.textFile('MIDS-MLS-HW-10.txt').flatMap(lambda x: x.strip().split()) \
    .filter(filterLowerCase) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(add) \
    .map(lambda (x, y): (y, x)) \
    .sortByKey(False) \
    .map(lambda (x, y): (y, x))
for word, count in wordCount.collect():
    print word, count

the 70
and 36
in 25
of 24
a 18
for 12
code 12
to 12
is 11
model 11
with 10
data 10
as 9
on 9
plot 9
this 8
your 8
each 7
domain 6
from 6
linear 6
training 5
following 5
clusters 5
it 5
regression 5
using 4
side 4
space 4
example 4
words 3
report 3
lazy 3
count 3
iteration 3
show 3
results 3
x 3
import 3
plots 3
by 3
that 3
an 3
document 3
provided 3
after 3
homework 2
every 2
word 2
notebook: 2
list 2
run 2
regression. 2
above 2
evaluation 2
available 2
https://www.dropbox.com/s/q85t0ytb9apggnh/kmeans_data.txt?dl=0 2
here 2
set. 2
comment 2
https://www.dropbox.com/s/3nsthvp8g2rrrdh/EM-Kmeans.ipynb?dl=0 2
use 2
you 2
function 2
surface 2
plots. 2
differences 2
original 2
heatmap 2
iterations. 2
iterations, 2
set 2
testing 2
label 2
vector 2
between 2
be 2
found 2
error 2
or 2
findings. 2
one 2
trajectory 2
more 2
work 2
up 2
can 2
at 2
output 2
kmeans_data.txt 2
how 2
any 2
repeat 2
points 2
decreasing 2
order 2
y) 1
sameModel 1
iterations 1
not 1
intuitoive 1
consists 1
space: 1
provid